In [1]:
import os
from lxml import etree as lxml
import zipfile, itertools
import io

In [2]:
def load_xml(z):
    with z.open("archive_manifest.xml", 'r') as f:
        tree = lxml.parse(f)
    return tree

def load_zip(file):
    return zipfile.ZipFile(file)

In [17]:
# documents to scan for artifacts
documents = [
    'PBS Bioreactor Market Requirements Document',
    '3.0.9 Test Plan',
    '3.0.9 Requirements',
    'ID00013A2 Master Project Document',
]

folders = [
    ("/Public/Public/Bioreactor Development/PBS Software/v3 Release/3.0.9", "/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9")
]

In [4]:
# load the data
thezip = load_zip("Helix Migration Test\ExportData.zip")
thetree = load_xml(thezip)
theroot = thetree.getroot()

In [5]:
s = set()
for e in theroot:
    s.add(e.tag)


{'defect',
 'folder',
 'link',
 'requirement',
 'requirement-document',
 'test-case'}

In [9]:
for e in theroot.xpath("folder"):
    break
for el in e:
    print(el.tag)

folder-name
folder-description
is-locked
permissions
task-board
folder-access
folder-path
folder-type
folder-notes
folder-public-details


In [44]:
s = set()
# load the data
thezip = load_zip("Helix Migration Test\ExportData.zip")
thetree = load_xml(thezip)
theroot = thetree.getroot()
for fldr in theroot.xpath("*/folder-path"):
    for old, new in folders:
        if old in fldr.text:
            fldr.text = fldr.text.replace(old, new)
            print(fldr.text)

/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Results
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/Bioreactor Control Software/v3.0+/3.0.9/Tests
/Public/Software Development/

In [56]:
xp = "*[./folder-path[contains(text(), %s)]]" % " | ".join("'%s'"%f for _, f in folders)
for item in theroot.xpath(xp):
   # print(item)
    if item.find("record-id") is None:
        print(item)

<Element folder at 0x211ffce9508>
<Element folder at 0x211ffd11348>
<Element folder at 0x211ffd11f48>


In [5]:

# map record id <-> artifact
themap = {}
for req in theroot.xpath("requirement"):
    themap[req.find("record-id").text] = req
    
# loop over documents and flag artifacts for import
# this is done to only import artifacts that were part of the document
art_seen = set()
to_import = []
docs = []

xp = "requirement-document[./summary/text()=(%s)]"% ", ".join('"%s"' % d for d in documents)

for doc in theroot.xpath(xp)
    docs.append(doc)
    for id in doc.xpath("document-tree-node/node-requirement-id/text()"):
        if id not in art_seen:
            art_seen.add(id)
            to_import.append(id)
            
# documents in the indicated folders are also included
for item in theroot.xpath("*/")
            
def docopy(q, z1, z2):
    for name, dst in q:
        with z1.open(name, 'r') as src:
            z2.writestr(dst, src.read())
zipq = []

newroot = lxml.Element(theroot.tag, theroot.attrib)
newroot.text = theroot.text
    
for rid in to_import:
    req = themap[rid]
    elem = lxml.SubElement(newroot, req.tag, req.attrib)
    elem.text = req.text
    elem.tail = req.tail
    to_replace = []
    for e in req:
        if e.tag == 'custom-field-value' and e.attrib['field-name'] != 'Release Notes':
            continue
        elif e.tag == 'record-id':
            continue
        elif e.tag == 'requirement-number':
            continue
        elif e.tag == 'attachment':
            a = lxml.SubElement(elem, e.tag, e.attrib)
            ofn = a.attrib['filespec']
            zipq.append(ofn)
        else:
            elem.append(e)

for doc in docs:
    newroot.append(doc)

nzip = zipfile.ZipFile("test\\test.zip", mode='w')

print("creating zip file")
newtree = lxml.ElementTree(newroot)
with io.BytesIO() as f:
    newtree.write(f, pretty_print=True, xml_declaration=True, standalone=False, encoding='UTF-8', doctype=b'<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">')
    data = f.getvalue()
    
nzip.writestr("archive_manifest.xml", data)
    
print("copying attachments")
zipq.docopy(dzip, nzip)
print("done")
nzip.close()

creating zip file
copying attachments
done


In [67]:
tzip = load_zip("TestCases.zip")
ttree = load_xml(tzip)
troot = ttree.getroot()

newroot = lxml.Element(troot.tag, troot.attrib)
newroot.text = troot.text

for t in troot.xpath("test-case[./folder-path[contains(text(), '3.0.9')]]"):
    newroot.append(t)
    for e in t.xpath("workflow-event/custom-field-value[@field-value='']"):
        e.getparent().remove(e)
    for e in t.xpath("custom-field-value[@field-value='']"):
        e.getparent().remove(e)
    
newtree = lxml.ElementTree(newroot)
with open("test\\out.xml", 'wb') as f:
    newtree.write(f, pretty_print=True, xml_declaration=True, standalone=False, encoding='UTF-8', 
                  doctype=b'<!DOCTYPE TestTrackData SYSTEM "TestTrackData.dtd">')